## Preparations

После перезагрузки компьютера, запустите следующие команды в терминале (пароль 1qaz!QAZ).
```bash
echo 1024 | sudo tee /sys/module/usbcore/parameters/usbfs_memory_mb
sudo cat /sys/module/usbcore/parameters/usbfs_memory_mb
```

В трех отдельных сессиях терминала запустите, если еще не запущено, по одной строке из
```bash
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh all
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh info
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh err
```
В этих сессиях будут отображаться (1) все сообщения с информацией о сканировании, (2) сообщения об основных этапах сканирования, (3) ошибки.

## Start the cameras

1. Запустите код в следующей клетке: он инициализирует камеры.
Проверьте по логам что все 6 камер инициализировались корректно.

In [ ]:
from pathlib import Path
import datetime
import sys

from ipywidgets import Box

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.utils.frame_widget import FrameWidget
from skrgbd.utils.logging import logger
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper


scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'
started_warmup_at = datetime.datetime.now()

scan_helper = ScanHelper(scans_root, logfile, trajectories='human_sphere', thread='cameras')
scan_helper.stop_warmup()

scan_helper.periphery.left_phone_ir.off()
scan_helper.periphery.right_phone_ir.off()

## Set the white balance of the cameras

Настройте баланс белого один раз после запуска тетрадки.
1. Выключите свет в комнате.
2. Запустите код в следующей клетке и следуйте инструкциям.

In [ ]:
robot = scan_helper.camera_trajectory_class()
robot.move_for_awb(.1)
scan_helper.set_white_balance()
scan_helper.stop_warmup()
robot.move_for_cam_settings(.1)
del robot

Проверьте правильность расположения ламп, и с помощью процедур ниже подберите настройки камер.

In [ ]:
scene_name = 'people_debug'

scan_helper.load_camera_settings(scene_name)  # настройки должны быть подобраны автоматически ранее
previews_dir = scan_helper.make_scene_dir(scene_name) / 'settings_previews'

## TIS

In [ ]:
camera = scan_helper.rig.tis_right

Выберите освещение вариант освещения для которого нужно подобрать настройки.

In [ ]:
# light, preset = ('flash', 'best')
# light, preset = ('flash', 'fast')
# light, preset = ('ambient', 'best')
# light, preset = ('ambient_low', 'fast')
# light, preset = ('hard_right_bottom_close', 'best')
# light, preset = ('hard_right_top_close', 'best')
# light, preset = ('hard_right_top_far', 'best')
# light, preset = ('hard_left_bottom_close', 'best')
# light, preset = ('hard_left_bottom_far', 'best')
# light, preset = ('hard_left_top_far', 'best')
# light, preset = ('soft_left', 'best')
# light, preset = ('soft_right', 'best')
# light, preset = ('soft_top', 'best')

logger.debug(f'ManualSettingsPicking: Pick settings for {light} at {preset}')
scan_helper.periphery.on_only(light)

Загрузите настройки подобранные автоматически в качестве отправной точки.

In [ ]:
camera.setup(light, preset)

Отобразите снимок с камеры.

In [ ]:
widget = FrameWidget(
    (lambda: camera.snap_frame()['image']),
    figsize=(16, 16), ticks=False
)
widget.update()
widget.image

Подберите настройки камеры. Снимок выше будет обновляться автоматически. Мониторьте адекватность настроек через логи.

In [ ]:
camera.set_exposure(camera.exposure * 1.5)
# camera.set_gain(camera.gain * .95)
widget.update()

После того как Вы подобрали настройки, сохраните их на диск.

In [ ]:
camera.exposure_presets[(light, preset)] = camera.exposure
camera.gain_presets[(light, preset)] = camera.gain

camera.copy_light_preset(('hard_right_top_close', 'best'), ('hard_left_top_close', 'best'))
scan_helper.rig.tis_left.exposure_presets = scan_helper.rig.tis_right.exposure_presets
scan_helper.rig.tis_left.gain_presets = scan_helper.rig.tis_right.gain_presets

camera.save_image(f'{previews_dir}/tis.{light}@{preset}', blocking=False)

scan_helper._save_camera_settings(scene_name)

Повторите для следующего варианта освещения для этой же камеры или переходите к следующей камере.

## Phone

In [ ]:
camera = scan_helper.rig.phone_right
camera.set_low_resolution()

Выберите освещение вариант освещения для которого нужно подобрать настройки.

In [ ]:
# light, preset = ('flash', 'best')
# light, preset = ('flash', 'fast')
# light, preset = ('ambient', 'best')
# light, preset = ('ambient_low', 'fast')
# light, preset = ('hard_right_bottom_close', 'best')
# light, preset = ('hard_right_top_close', 'best')
# light, preset = ('hard_right_top_far', 'best')
# light, preset = ('hard_left_bottom_close', 'best')
# light, preset = ('hard_left_bottom_far', 'best')
# light, preset = ('hard_left_top_far', 'best')
# light, preset = ('soft_left', 'best')
# light, preset = ('soft_right', 'best')
# light, preset = ('soft_top', 'best')

logger.debug(f'ManualSettingsPicking: Pick settings for {light} at {preset}')
scan_helper.periphery.on_only(light)

Загрузите настройки подобранные автоматически в качестве отправной точки.

In [ ]:
camera.setup(light, preset)

Отобразите снимок с камеры.

In [ ]:
widget = FrameWidget(
    (lambda: camera.get_photo()),
    figsize=(8, 8), ticks=False
)
widget.update()
widget.image

Подберите настройки камеры. Снимок выше будет обновляться автоматически. Мониторьте адекватность настроек через логи.

In [ ]:
# camera.set_exposure(camera.exposure * 1.5)
# camera.set_iso(camera.iso * 1.05)
widget.update()

После того как Вы подобрали настройки, сохраните их на диск.

In [ ]:
camera.exposure_presets[(light, preset)] = camera.exposure
camera.iso_presets[(light, preset)] = camera.iso

camera.copy_light_preset(('hard_right_top_close', 'best'), ('hard_left_top_close', 'best'))
scan_helper.rig.phone_left.exposure_presets = scan_helper.rig.phone_right.exposure_presets
scan_helper.rig.phone_left.iso_presets = scan_helper.rig.phone_right.iso_presets

camera.save_light_picking_frame(f'{previews_dir}/phone.{light}@{preset}.jpg')

scan_helper._save_camera_settings(scene_name)

Повторите для следующего варианта освещения для этой же камеры или переходите к следующей камере.

In [ ]:
camera.set_high_resolution()

## RealSense RGB

In [ ]:
camera = scan_helper.rig.realsense

Выберите освещение вариант освещения для которого нужно подобрать настройки.

In [ ]:
# light, preset = ('flash', 'best')
# light, preset = ('flash', 'fast')
# light, preset = ('ambient', 'best')
# light, preset = ('ambient_low', 'fast')
# light, preset = ('hard_right_bottom_close', 'best')
# light, preset = ('hard_right_top_close', 'best')
# light, preset = ('hard_right_top_far', 'best')
# light, preset = ('hard_left_bottom_close', 'best')
# light, preset = ('hard_left_bottom_far', 'best')
# light, preset = ('hard_left_top_far', 'best')
# light, preset = ('soft_left', 'best')
# light, preset = ('soft_right', 'best')
# light, preset = ('soft_top', 'best')

logger.debug(f'ManualSettingsPicking: Pick settings for {light} at {preset}')
scan_helper.periphery.on_only(light)

Загрузите настройки подобранные автоматически в качестве отправной точки.

In [ ]:
camera.setup(light, preset)

Отобразите снимок с камеры.

In [ ]:
widget = FrameWidget(
    (lambda: camera.snap_frame()['image']),
    figsize=(8, 8), ticks=False
)
widget.update()
widget.image

Подберите настройки камеры. Снимок выше будет обновляться автоматически. Мониторьте адекватность настроек через логи.

In [ ]:
# camera.set_rgb_exposure(camera.rgb_exposure * .9)
# camera.set_rgb_gain(camera.rgb_gain * 3)
widget.update()

После того как Вы подобрали настройки, сохраните их на диск.

In [ ]:
camera.exposure_presets[(light, preset)] = camera.rgb_exposure
camera.gain_presets[(light, preset)] = camera.rgb_gain

camera.copy_light_preset(('hard_right_top_close', 'best'), ('hard_left_top_close', 'best'))
camera.save_image(f'{previews_dir}/realsense.{light}@{preset}', modalities={'image'}, blocking=False, compressed=True)

scan_helper._save_camera_settings(scene_name)

Повторите для следующего варианта освещения для этой же камеры или переходите к следующей камере.

## RealSense IR

In [ ]:
camera = scan_helper.rig.realsense

Выберите освещение вариант освещения для которого нужно подобрать настройки.

In [ ]:
# light, preset = ('flash', 'best')
# light, preset = ('flash', 'fast')
# light, preset = ('ambient', 'best')
# light, preset = ('ambient_low', 'fast')
# light, preset = ('hard_right_bottom_close', 'best')
# light, preset = ('hard_right_top_close', 'best')
light, preset = ('hard_right_top_far', 'best')
# light, preset = ('hard_left_bottom_close', 'best')
# light, preset = ('hard_left_bottom_far', 'best')
# light, preset = ('hard_left_top_far', 'best')
# light, preset = ('soft_left', 'best')
# light, preset = ('soft_right', 'best')
# light, preset = ('soft_top', 'best')

logger.debug(f'ManualSettingsPicking: Pick settings for {light} at {preset}')
scan_helper.periphery.on_only(light)

Загрузите настройки подобранные автоматически в качестве отправной точки.

In [ ]:
camera.setup(light, preset)

Отобразите снимок с камеры.

In [ ]:
ir_widget = FrameWidget(
    (lambda: camera.snap_frame()['ir']),
    figsize=(16, 16), ticks=False, cmap='gray', vmin=0, vmax=1
)
ir_widget.update()
depth_widget = FrameWidget(
    (lambda: camera.snap_frame()['depth']),
    figsize=(16, 16), ticks=False, cmap='plasma_r', vmin=500, vmax=1200
)
depth_widget.update()

images = [ir_widget.image, depth_widget.image]
for image in images:
    image.width = '450px'
    image.layout.object_fit = 'contain'
widget = Box(images)
widget

Подберите настройки камеры. Снимок выше будет обновляться автоматически. Мониторьте адекватность настроек через логи.

In [ ]:
# camera.set_ir_exposure(camera.ir_exposure * 1.1)
# camera.set_ir_gain(camera.ir_gain)
# camera.set_laser_power(camera.laser_power + 30)
# camera.set_laser_power(0)
ir_widget.update()
depth_widget.update()

После того как Вы подобрали настройки, сохраните их на диск.

In [ ]:
camera.ir_exposure_presets[(light, preset)] = camera.ir_exposure
camera.ir_gain_presets[(light, preset)] = camera.ir_gain

camera.copy_light_preset(('hard_right_top_close', 'best'), ('hard_left_top_close', 'best'))
camera.save_image(f'{previews_dir}/realsense.{light}@{preset}', modalities={'ir', 'ir_right'}, blocking=False, compressed=True)

scan_helper._save_camera_settings(scene_name)

Повторите для следующего варианта освещения для этой же камеры или переходите к следующей камере.

## Stop the cameras

In [14]:
scan_helper.stop()
robot = scan_helper.camera_trajectory_class()
robot.rest(.1)